In [1]:


from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI

from agent.utils.loader import load_processed_data

_ = load_dotenv(find_dotenv())

In [4]:
meta_info = {
	"dataset_name": 'hotpot_qa',
	"mode": "self-improve",
	"base_mode": "cot",
	"model": "gpt-4o-mini-2024-07-18",
	"num_samples": -1,
	"top_p": 0.95,
	"temperature": 0,
	"seed": 42,
	"batch_size": 100
}
assert meta_info["mode"] == "self-improve"
assert meta_info["dataset_name"] in ["hotpot_qa", "trivia_qa", "ambig_qa"], "Invalid dataset name"

if meta_info["base_mode"] == "cot":
	processed_data_path = f"../../../output/inference/{meta_info["model"]}/{meta_info["dataset_name"]}/cot/num_samples_1000_top_p_0.95_temperature_0_seed_42.jsonl"
else:
	processed_data_path = f"../../../data/processed_data/{meta_info["dataset_name"]}.jsonl"

dataset = load_processed_data(dataset_name=meta_info["dataset_name"], file_path=processed_data_path)
model = ChatOpenAI(
	model_name=meta_info["model"],
	top_p=meta_info["top_p"],
	temperature=meta_info["temperature"],
	seed=meta_info["seed"],
	openai_api_base="https://api.chsdw.top/v1"
)

if meta_info["num_samples"] > 0:
	dataset = dataset.select(range(meta_info["num_samples"]))
print(dataset[10])



Generating train split: 0 examples [00:00, ? examples/s]

{'context': '', 'question': 'What is the name of the fight song of the university whose main campus is in Lawrence, Kansas and whose branch campuses are in the Kansas City metropolitan area?', 'answer': ['Kansas Song'], 'generation': 'A: Let\'s think step by step. The university with a main campus in Lawrence, Kansas, and branch campuses in the Kansas City metropolitan area is the University of Kansas. The fight song of the University of Kansas is "I\'m a Jayhawk." FINAL ANSWER: "I\'m a Jayhawk."', 'prediction': '"I\'m a Jayhawk."'}


In [8]:

from typing_extensions import TypedDict


class State(TypedDict):
	question: str
	guidance: str
	initial_answer: str
	prediction: str
	answer: str

In [9]:
from langchain_core.prompts import ChatPromptTemplate

guidance_prompt = ChatPromptTemplate.from_messages(
	[
		(
			"system",
			"You are a question planner and error prone points identifier. Given a question or problem, your job is to come up with a step by step plan, and you should also identify the most error-prone points for each step, following them closely behind each step. The plan and the error prone points will then be used to guide the selection of subsequent tools and the corresponding tool inputs. The tool results should always be considered as reliable. Do not add any superfluous steps. Make sure that each step has all the information needed - do not skip steps. You should focus on the logic of how to solve the problem, rather than actually solving it."
		),
		(
			"user",
			"Question: {question}"
		)
	])
guidance_generator = guidance_prompt | model


async def guidance_step(state: State) -> State:
	assert state["question"] is not None, "Question is required"
	question = state["question"]
	guidance = await guidance_generator(question=question)
	state["guidance"] = guidance
	return state

NameError: name 'model' is not defined

In [11]:
from langchain_community.utilities.wikidata import WikidataAPIWrapper
from langgraph.prebuilt import create_react_agent
from agent.utils.tools import GoogleSearchTool, GoogleKnowledgeGraphTool, WikidataTool, WikipediaTool

google_search = GoogleSearchTool()
google_knowledge_graph = GoogleKnowledgeGraphTool()
wikidata = WikidataTool(api_wrapper=WikidataAPIWrapper())
wikipedia = WikipediaTool()
tools = [google_search, google_knowledge_graph, wikipedia, wikidata]

critique_prompt = "You are a reactive agent. Given a question or problem, your job is to select the most appropriate tool to answer the question or solve the problem. You should consider the guidance provided by the question planner and error prone points identifier, and the tool results are reliable."

critique_agent = create_react_agent(tools=tools, model=model, state_modifier=critique_prompt)

In [12]:
inputs = {"messages": [
	("user", "What government position was held by the woman who portrayed Corliss Archer in the film Kiss and Tell?")]}
result = critique_agent.invoke(inputs, stream_mode="values")
print(result)

{'messages': [HumanMessage(content='What government position was held by the woman who portrayed Corliss Archer in the film Kiss and Tell?', additional_kwargs={}, response_metadata={}, id='20c58d19-f24b-40f2-b182-66ead393cf20'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_oG7n4z3leFrItgUsQzrrvrBG', 'function': {'arguments': '{"query":"Corliss Archer Kiss and Tell"}', 'name': 'wikipedia'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 327, 'total_tokens': 346, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_04751d0b65', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7fd68336-a969-4632-9fec-edfbea8a3ea5-0', tool_calls=[{'name': 'wikipedia', 'args': {'query': 'Corliss Archer Kiss and Tell'}, 'id': 'call_oG7n4z3leFrItgUsQzrrvrBG', 'type': 'tool_call'}], usage_metadata={'input_tokens': 327, 'output_tokens': 19, '

In [13]:
print(result.keys())

dict_keys(['messages'])
